# Exercício de Sugestão de Compras utilizando Word2Vec
### Aluno: João Paulo Brum
### Disciplina: Deep Learning

In [1]:
import warnings
warnings.filterwarnings("ignore")

# Passo 1: Baixar Dados

Aqui usaremos uma planilha do Microsoft Excel com dados de compras de usuários em um sistema de varejo. O nome do arquivo é `retail.xlsx`.

In [2]:
!gdown https://drive.google.com/uc?id=1NK-2z0l-qTplDJJ2SHpTVBGRP3zWAK-n

Downloading...
From: https://drive.google.com/uc?id=1NK-2z0l-qTplDJJ2SHpTVBGRP3zWAK-n
To: /content/retail.xlsx
23.7MB [00:00, 57.2MB/s]


Usaremos a biblioteca `pandas` para fazer a leitura dos dados

In [3]:
import pandas as pd

Aqui fazemos a leitura dos dados para um objeto `DataFrame` do `pandas`.

In [4]:
# Esta linha de código pode demorar cerca de 1 min para rodar
df = pd.read_excel('retail.xlsx')

Agora podemos dar uma observada nos dados para entender como estão organizados.

In [5]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


A tabela acima mostra a estrutura da planilha. Podemos observar os campos:

- **InvoiceNo**: Este é um identificador único para cada compra.
- **StockCode**: Identificador único para cada produto.
- **Description**: Descrição do produto.
- **Quantidade**: Quantidade daquele produto, naquela compra.
- **InvoiceDate**: Dia e hora da compra.
- **CustomerID**: Identificador único do cliente.

In [6]:
df.shape

(541909, 8)

Acima vemos que essa base de dados possui 541.909 registros. Cada registro representa uma linha, ou seja, um produto comprado (cuja quantidade pode ser maior que 1). Diferentes linhas podem representar diferentes itens de uma mesma compra.

# Passo 2: Remoção de Dados Nulos

Neste passo vamos remover da base de dados os dados relativos a compras de produtos onde algum dos dados de interesse estejam faltando.

Começamos observando a quantidade de dados nulos

In [7]:
df.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

Acima podemos observar há 1.454 registros sem a descrição do produto, e 135.080 registros sem a identificação do cliente comprador.

No código abaixo descartaremos esses dados que aparecem com algum campo nulo.

In [8]:
# Remove dados nulos
df.dropna(inplace=True)
# Verifica novamente
df.isnull().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

# Passo 3: Dicionário de Produtos

In [9]:


# Separa apenas as colunas de código de produto e descrição
products = df[["StockCode", "Description"]]

# Transforma os StockCodes em strings
products['StockCode'] = products['StockCode'].astype(str)

# Remove duplicados
products.drop_duplicates(inplace=True, subset='StockCode', keep="last")

# create product-ID and product-description dictionary
products_dict = products.groupby('StockCode')['Description'].apply(list).to_dict()

In [10]:
# Teste do dicionário
products_dict['84854']

['GIRLY PINK TOOL SET']

# Passo 4: Preparação dos Dados

Neste passo vamos preprarar os dados transformando o histórico de compras de cada consumidor numa espécie de "frase", onde cada "palavra" é um produto comprado.

Começamos convertendo os códigos de produto (coluna **StockCode**) para string, para usar como "palavras" no treinamento de um modelo word2vec no `gensim` mais tarde.

In [11]:
df['StockCode'] = df['StockCode'].astype(str)

Agora vamos confirmar quantos clientes únicos temos nessa base de dados

In [12]:
# A linha abaixo cria uma lista coletando os ids
# da coluna CustomerID, selecionando apenas ids
# únicos (não recolhe ids repetidos)
customers = df["CustomerID"].unique().tolist()
len(customers)

4372

Acima verificamos que há 4.372 clientes na base de dados. Para cada um desses clientes vamos verificar o histórico de compras, criando 4.372 sequências de compras.

# Passo 4: Separação em treino e validação

Neste passo vamos preprarar separar os dados em dados de treino e de validação.


No código abaixo embaralhamos a ordem dos ids dos clientes na lista `customers`.

In [13]:
import numpy as np
import random
random.shuffle(customers)

Agora vamos separar dados de treinamento e de validação

In [14]:
# Calcula a quantidade de clientes que usaremos
# para treinamento
train_size = int(0.9 * len(customers))

# Separa os consumidores em duas listas: uma para
# treinamento e a outra para validação
customers_train = customers[:train_size]
customers_val = customers[train_size:len(customers)]

# Baseados nessa separação acima, separamos os dados
df_train = df[df['CustomerID'].isin(customers_train)]
df_val = df[df['CustomerID'].isin(customers_val)]

# Passo 5: Cria um histórico de compras de cada cliente

Com essas listas de clientes criamos abaixo as sequências de compras de acordo com os históricos de cada cliente.

In [15]:
# Esse módulo serve para mostrar uma barra de progresso
def compile_orders(customers, df):
  ''' Essa função coleta todas compras do histódico
      de cada cliente. O parâmetro customers é a lista
      de ids de clientes e o parâmetro df é o objeto
      DataFrame do pandas com os dados de cada compra.
      O valor retornado é uma lista de listas, onde cada
      lista interna contém a seguência de códigos de produto
      de cada compra, na ordem que se apresentava no
      histórico.
  '''
  orders = []
  for customer in customers:
    order = df[df['CustomerID'] == customer]['StockCode'].tolist()
    orders.append(order)
  return orders

In [16]:
# Aqui separamos as listas de listas de compras. Este código
# demora cerca de 1 minuto para rodar
orders_train = compile_orders(customers_train, df_train)
orders_val = compile_orders(customers_val, df_val)

In [17]:
# Mostra o cliente e a lista de compra do mesmo
print(' Cliente: ' + str(customers[0]))
print(' Comprou: ' + str(orders_train[0]))

 Cliente: 13926.0
 Comprou: ['22470', '22694', '22910', '23322', '84945', '21498', '20750', '23503', '21630', '46000M', '84078A']


# Passo 6: Importa o GENSIN para criar o modelo Word2Vec

In [18]:
# Importa o módulo GENSIN para embedding word2vec
import gensim

In [19]:
# Cria o modelo do word2vec
model = gensim.models.Word2Vec(orders_train, size = 50, window = 8, min_count = 20, workers = 10, iter = 20)

In [34]:
# Função que encontra produtos similares no modelo de word2vec e printa para o usuário
def similar_products(code):
  print('Product description from the code: ' + str(products_dict[str(code)][0]))

  list_of_similar = model.wv.most_similar(positive=str(code))
  print('\nSimilar products: ')
  for stock_code, similarity in list_of_similar:
    print('Similarity: ' + str(similarity*100) + '% ', ' \t Product: ' + str(products_dict[str(stock_code)][0]))


#Passo 7: Cria sugestões à partir de Listas

In [35]:
# Teste da Função que retorna produtos similares a partir do código
similar_products(21733)

Product description from the code: RED HANGING HEART T-LIGHT HOLDER

Similar products: 
Similarity: 78.16364169120789%   	 Product: PINK HANGING HEART T-LIGHT HOLDER
Similarity: 63.269299268722534%   	 Product: CREAM HANGING HEART T-LIGHT HOLDER
Similarity: 47.1172958612442%   	 Product: CREAM HEART CARD HOLDER
Similarity: 46.76222503185272%   	 Product: HANGING METAL STAR LANTERN
Similarity: 44.54137980937958%   	 Product: GLASS STAR FROSTED T-LIGHT HOLDER
Similarity: 44.16430592536926%   	 Product: ANT WHITE WIRE HEART SPIRAL
Similarity: 43.30875277519226%   	 Product: HANGING METAL HEART LANTERN
Similarity: 40.46420156955719%   	 Product: HEART OF WICKER LARGE
Similarity: 39.573997259140015%   	 Product: COLOURED GLASS STAR T-LIGHT HOLDER
Similarity: 38.83829116821289%   	 Product: PINK HEART SHAPE EGG FRYING PAN


In [51]:
# Função que à partir de uma lista de compras, calcula o vetor médio e retorna uma lista de sugestões para a lista de compras

def list_suggestion(shopping_list):
  
  print('Shopping List: ')
  v = 0
  for product in shopping_list:
    if product in model:
      print(str(products_dict[str(product)][0]))
      v = v + model[str(product)]
      v = v/len(shopping_list)   
      similars = model.similar_by_vector(v)
      products_similars = [i[0] for i in similars]
      similarities = [i[1] for i in similars]
    
  
  # Check se há produtos sugeridos na lista de compras e remove
  duplicados =list(set(shopping_list).intersection(products_similars))
  for duplicado in duplicados:
    products_similars.remove(duplicado)

  # Printa os produtos sugeridos e as respectivas similaridades com a lista
  print('\nSuggested Products: ')
  for product, similarity in zip(products_similars, similarities):
    print('Similarity: ' + str(similarity*100) + '%' + '\t Product: ' + str(products_dict[product][0]))
  

In [53]:
# Pega 5 lista de clientes aleatórios e sugere uma lista de produtos
for i in range(5):
  k = np.random.randint(len(orders_val))
  print('\nClient number ' + str(k) + ' from validation set')
  list_suggestion(orders_val[k])


Client number 123 from validation set
Shopping List: 
JAZZ HEARTS ADDRESS BOOK
PINK VINTAGE PAISLEY PICNIC BAG
BLUE CHARLIE+LOLA PERSONAL DOORSIGN
RED CHARLIE+LOLA PERSONAL DOORSIGN
CHARLIE & LOLA WASTEPAPER BIN FLORA
CHARLIE & LOLA WASTEPAPER BIN BLUE
LAVENDER SCENTED FABRIC HEART
GINGHAM HEART DECORATION
LARGE WHITE HEART OF WICKER
CREAM HANGING HEART T-LIGHT HOLDER
JUMBO BAG PAISLEY PARK
JUMBO BAG PINK POLKADOT

Suggested Products: 
Similarity: 99.74992275238037%	 Product: JUMBO SHOPPER VINTAGE RED PAISLEY
Similarity: 83.766108751297%	 Product: JUMBO BAG STRAWBERRY
Similarity: 83.53216648101807%	 Product: JUMBO STORAGE BAG SUKI
Similarity: 83.4604024887085%	 Product: JUMBO STORAGE BAG SKULLS
Similarity: 82.9266905784607%	 Product: JUMBO  BAG BAROQUE BLACK WHITE
Similarity: 81.09998106956482%	 Product: JUMBO BAG TOYS 
Similarity: 80.62930107116699%	 Product: JUMBO BAG OWLS
Similarity: 80.61365485191345%	 Product: JUMBO BAG SCANDINAVIAN BLUE PAISLEY
Similarity: 79.64452505111694%	 Pr